# Manutenzione preventiva - DoS Attacks

In [1]:
import pandas as pd
import json as json
import ipaddress as ip 
import numpy as np
import warnings
import functions_notebook_new as fn
from datetime import timedelta



warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
### costanti
# clean captures
clean_capture = 'captures\\captures1_v2\\clean\\eth2dump-clean-0,5h_1.json'

# ping flooding captures 1 min
pf_capture_1 = 'captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS15m-0,5h_1.json'

levels_of_interest = ['frame', 'eth', 'tcp', 'arp', 'udp', 'modbus', 'icmp', 'ip']

### variabili

In [3]:
### estrazione dati
# packets clean capture
packets_cc = fn.load_packets_from_file(clean_capture)

# packets attack captures (es. packets ping flooding attack 1m per cattura di 0.5 = pkt_pf_1m_05h)
pkt_pf_1m_05h = fn.load_packets_from_file(pf_capture_1)

In [4]:
### features
# clean features
features_cc = fn.extract_features_from_packets(packets_cc, levels_of_interest)

# ping flooding attack features
features_pf_1m_05h = fn.extract_features_from_packets(pkt_pf_1m_05h, levels_of_interest)

In [5]:
df_cc = pd.DataFrame(features_cc)
df = pd.DataFrame(features_pf_1m_05h)

In [6]:
features_cc.clear()
features_pf_1m_05h.clear()

In [7]:
fn.calcolo_features_binarie(df_cc)
fn.calcolo_features_binarie(df)

In [8]:
df_cc['frame.frame.time_utc'] = pd.to_datetime(df_cc['frame.frame.time_utc'])
df['frame.frame.time_utc'] = pd.to_datetime(df['frame.frame.time_utc'])

In [9]:
df_cc.sort_values(by='frame.frame.time_utc')
df.sort_values(by='frame.frame.time_utc')

,frame.frame.encap_type,frame.frame.time,frame.frame.time_utc,frame.frame.time_epoch,frame.frame.offset_shift,frame.frame.time_delta,frame.frame.time_delta_displayed,frame.frame.time_relative,frame.frame.number,frame.frame.len,...,mbtcp,ipv6,tcp,icmp,llc,stp,ip,modbus,eth,arp
0,1,"May 21, 2018 13:12:37.348723000 ora legale Eur...",2018-05-21 11:12:37.348723+00:00,1526901157.348723000,0.000000000,0.000000000,0.000000000,0.000000000,1,60,...,0,0,0,0,0,0,0,0,1,1
1,1,"May 21, 2018 13:12:37.350527000 ora legale Eur...",2018-05-21 11:12:37.350527+00:00,1526901157.350527000,0.000000000,0.001804000,0.001804000,0.001804000,2,64,...,0,0,0,0,0,0,0,0,1,1
2,1,"May 21, 2018 13:12:37.350544000 ora legale Eur...",2018-05-21 11:12:37.350544+00:00,1526901157.350544000,0.000000000,0.000017000,0.000017000,0.001821000,3,60,...,0,0,1,0,0,0,1,0,1,0
3,1,"May 21, 2018 13:12:37.360801000 ora legale Eur...",2018-05-21 11:12:37.360801+00:00,1526901157.360801000,0.000000000,0.010257000,0.010257000,0.012078000,4,60,...,0,0,1,0,0,0,1,0,1,0
4,1,"May 21, 2018 13:12:37.360805000 ora legale Eur...",2018-05-21 11:12:37.360805+00:00,1526901157.360805000,0.000000000,0.000004000,0.000004000,0.012082000,5,60,...,0,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197773,1,"May 21, 2018 13:42:36.345657000 ora legale Eur...",2018-05-21 11:42:36.345657+00:00,1526902956.345657000,0.000000000,0.001220000,0.001220000,1798.996934000,197774,60,...,0,0,1,0,0,0,1,0,1,0
197774,1,"May 21, 2018 13:42:36.345673000 ora legale Eur...",2018-05-21 11:42:36.345673+00:00,1526902956.345673000,0.000000000,0.000016000,0.000016000,1798.996950000,197775,60,...,0,0,1,0,0,0,1,0,1,0
197775,1,"May 21, 2018 13:42:36.345854000 ora legale Eur...",2018-05-21 11:42:36.345854+00:00,1526902956.345854000,0.000000000,0.000181000,0.000181000,1798.997131000,197776,60,...,0,0,1,0,0,0,1,0,1,0
197776,1,"May 21, 2018 13:42:36.354401000 ora legale Eur...",2018-05-21 11:42:36.354401+00:00,1526902956.354401000,0.000000000,0.008547000,0.008547000,1799.005678000,197777,60,...,0,0,1,0,0,0,1,0,1,0


In [10]:
start_time = df['frame.frame.time_utc'].iloc[0]
end_time=start_time+timedelta(seconds=298.8)
end_time2 = end_time + timedelta(seconds=900)
end_time

Timestamp('2018-05-21 11:17:36.148723+0000', tz='UTC')

In [11]:
df_cc1 = df[(df['frame.frame.time_utc'] >= start_time) & (df['frame.frame.time_utc'] < end_time)].copy()
df_attack = df[(df['frame.frame.time_utc'] >= end_time) & (df['frame.frame.time_utc'] <= end_time2)].copy()
df_cc2 = df[(df['frame.frame.time_utc'] > end_time2)].copy()

,frame.frame.encap_type,frame.frame.time,frame.frame.time_utc,frame.frame.time_epoch,frame.frame.offset_shift,frame.frame.time_delta,frame.frame.time_delta_displayed,frame.frame.time_relative,frame.frame.number,frame.frame.len,...,mbtcp,ipv6,tcp,icmp,llc,stp,ip,modbus,eth,arp
2951,1,"May 21, 2018 13:17:36.233521000 ora legale Eur...",2018-05-21 11:17:36.233521+00:00,1526901456.233521000,0.000000000,0.164967000,0.164967000,298.884798000,2952,60,...,0,0,0,1,0,0,1,0,1,0
2952,1,"May 21, 2018 13:17:36.234592000 ora legale Eur...",2018-05-21 11:17:36.234592+00:00,1526901456.234592000,0.000000000,0.001071000,0.001071000,298.885869000,2953,60,...,0,0,0,0,0,0,0,0,1,1
2953,1,"May 21, 2018 13:17:36.283539000 ora legale Eur...",2018-05-21 11:17:36.283539+00:00,1526901456.283539000,0.000000000,0.048947000,0.048947000,298.934816000,2954,60,...,0,0,0,1,0,0,1,0,1,0
2954,1,"May 21, 2018 13:17:36.333570000 ora legale Eur...",2018-05-21 11:17:36.333570+00:00,1526901456.333570000,0.000000000,0.050031000,0.050031000,298.984847000,2955,60,...,0,0,0,1,0,0,1,0,1,0
2955,1,"May 21, 2018 13:17:36.383612000 ora legale Eur...",2018-05-21 11:17:36.383612+00:00,1526901456.383612000,0.000000000,0.050042000,0.050042000,299.034889000,2956,60,...,0,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190951,1,"May 21, 2018 13:32:36.104433000 ora legale Eur...",2018-05-21 11:32:36.104433+00:00,1526902356.104433000,0.000000000,0.000079000,0.000079000,1198.755710000,190952,60,...,0,0,0,1,0,0,1,0,1,0
190952,1,"May 21, 2018 13:32:36.104434000 ora legale Eur...",2018-05-21 11:32:36.104434+00:00,1526902356.104434000,0.000000000,0.000001000,0.000001000,1198.755711000,190953,60,...,0,0,0,1,0,0,1,0,1,0
190953,1,"May 21, 2018 13:32:36.104447000 ora legale Eur...",2018-05-21 11:32:36.104447+00:00,1526902356.104447000,0.000000000,0.000013000,0.000013000,1198.755724000,190954,60,...,0,0,0,1,0,0,1,0,1,0
190954,1,"May 21, 2018 13:32:36.104485000 ora legale Eur...",2018-05-21 11:32:36.104485+00:00,1526902356.104485000,0.000000000,0.000038000,0.000038000,1198.755762000,190955,60,...,0,0,0,1,0,0,1,0,1,0


In [68]:
df_cc['frame.frame.time_utc'] = pd.to_datetime(df_cc['frame.frame.time_utc'], unit='s')

df_features = df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s')).agg({
    # Max e min di feature binarie
	'eth': ['max','min'],
    'ip': ['max','min'],
    'tcp': ['max','min'],
    'arp': ['max','min'],
    'udp': ['max','min'],
    ''
    #'icmp': ['max','min'], (non è presente nella cattura clean)
    'mbtcp': ['max','min'],
    # Feature casuale per conteggio del numero di pacchetti per raggruppamento
    'frame.frame.ignored': ['count'],
    # Numero di IP unici per intervallo di tempo
    'ip.ip.src': ['nunique'],
    'ip.ip.dst': ['nunique'],
    #'': 'nunique'  # Numero di protocolli unici per intervallo di tempo
}).reset_index()

ValueError: Item wrong length 1 instead of 197778.